# Notebook 1: Primeira analise & Tratamento de dados

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from collections import Counter

In [2]:
#padroniza valores pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
df = pd.read_csv("scanner_data.csv")

### Visualizando e entendendo base de dados

In [4]:
df.head(3)

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,1,02/01/2016,2547,1,X52,0EM7L,1.00,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.00,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.00,6.35


In [5]:
# sem nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131706 entries, 0 to 131705
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      131706 non-null  int64  
 1   Date            131706 non-null  object 
 2   Customer_ID     131706 non-null  int64  
 3   Transaction_ID  131706 non-null  int64  
 4   SKU_Category    131706 non-null  object 
 5   SKU             131706 non-null  object 
 6   Quantity        131706 non-null  float64
 7   Sales_Amount    131706 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 8.0+ MB


In [3]:
# padronizando coluna date com pandas
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

In [5]:
# dados sao do periodo de 1 ano
df.Date.max()

Timestamp('2016-12-31 00:00:00')

#### Mini analise carrinhos

In [6]:
# 1.1 64k de transações, o que equivale a pouco mais de dois produtos por compra
# 1.2 maiores carrinhos ainda nao sao grandes, 21 produtos maximo
df.Transaction_ID.value_counts()

1348     21
19098    21
42295    20
39032    19
28731    19
         ..
28442     1
28443     1
28447     1
28448     1
32342     1
Name: Transaction_ID, Length: 64682, dtype: int64

In [16]:
print("Total de produtos comprados:", df.shape[0])
print("Total de carrinhos:", len(df.Transaction_ID.unique()))
print("Media de produtos por carrinho: %.2f" % (df.shape[0]/len(df.Transaction_ID.unique())))
print('%.2f%% das compras sao ate 5 produtos' % (((df.Transaction_ID.value_counts() < 5).value_counts().iloc[0] / len(df.Transaction_ID.unique())) *100))

Total de produtos comprados: 131706
Total de carrinhos: 64682
Media de produtos por carrinho: 2.04
92.81% das compras sao ate 5 produtos


#### Mini analise clientes

In [7]:
# 22k de clientes, tendo media por volta de 6 compras por cliente
df.Customer_ID.value_counts()

1660     228
1665     222
17104    218
1685     191
16905    179
        ... 
5906       1
21282      1
20192      1
21386      1
11733      1
Name: Customer_ID, Length: 22625, dtype: int64

#### Mini analise produtos

In [170]:
# 5242 produtos disponiveis caso todos produtos tenham sido vendidos pelo menos uma vez naquele ano
df.SKU.value_counts()

UNJKW    2007
COWU2     791
OV1P9     737
M6J9W     698
C6TXL     689
         ... 
DV08L       1
NXHF1       1
4YL6Q       1
T6EUH       1
9E9GV       1
Name: SKU, Length: 5242, dtype: int64

##### categorias dos produtos

In [8]:
df.SKU_Category.value_counts()

N8U    10913
R6E     5099
LPF     5062
P42     4836
U5F     4570
       ...  
M8H        3
U3N        2
QON        1
2JO        1
OTK        1
Name: SKU_Category, Length: 187, dtype: int64

In [63]:
# 1.1 Loja possui uma categoria chefe que destoa das outras com o dobro de vendas em relaçao a segunda, mas que porem possui
# ... menos de 10% das compras da loja, o que leva a crer que a loja nao depende exclusivamente dessa nem de nenhuma outra 
# ... categoria para sua essencialidade(claro, sem desprezar que ainda é a categoria mais vendida da loja)
df.SKU_Category.value_counts()[:10]

N8U    10913
R6E     5099
LPF     5062
P42     4836
U5F     4570
0H2     4500
IEV     4305
FEW     3164
29A     3033
H15     3011
Name: SKU_Category, dtype: int64

In [67]:
# 1.1 Apenas 15 das 187 categorias tem menos de 10 vendas, o que leva a crer 
# ... que a grande maioria das categorias sao relevantes para a loja
(df.SKU_Category.value_counts() < 10).value_counts()

False    172
True      15
Name: SKU_Category, dtype: int64

# As analises anteriores sao instrutivas mas faltam uma informaçao muito importante que é a receita gerada dos produtos vendidos em relaçao a receita total, para isso a base de dados sera reorganizada e mais a frente refeita a analise 

### Criando base de dados de produtos

In [ ]:
# 1.1 Base de dados carece de informaçao de estoque
# 1.2 Preços possuem variaçoes cujos alguns podem ser facilmente classificados como descontos ou aumento/diminuiçao de preço do
# ... produto, porem por vezes fica dificil o entendimento, entao optei por usar uma media de faturamento por unidades vendidas,
# ... ou seja usando um preço medio ao inves de um preço fixo de produto
# 1.3 Uma facilitaçao interessante seria colocar em cada linha/row (que contem as informaçoes separadas dos produtos) adicionar
# ... uma coluna contendo um dicionario contendo os produtos que ja foram vendidos com o produto em questao e quantas vezes
# ... esses produtos foram comprados em conjunto(como em um algoritimo apriori), entao fiz uma tentativa de como essa coluna 
# ... poderia ficar no pior cenario possivel, que é provavelmente com o produto que mais vezes foi vendido que deve possuir a
# ... maior quantidade de produtos que foram vendidos juntamente a ele, essa tentativa esta feita no final do notebook e foi
# ... concluido que seria incompativel com a intençao da base de dados e que se o algoritimo for necessario no futuro entao
# ... sera feito em um lugar separado

In [70]:
products = pd.DataFrame(columns=['SKU', 'SKU_Category', "Units_Sold", "Total_Amount",'Unit_Amount_Average'])
products

,SKU,SKU_Category,Units_Sold,Total_Amount,Unit_Amount_Average


In [71]:
%%time
for sku, data in df.groupby(['SKU']):
    Unit_Amount_Average = data.Sales_Amount.sum()/data.Quantity.sum()
    products.loc[len(products.index)] = [sku, data.SKU_Category.iloc[0], data.Quantity.sum(), data.Sales_Amount.sum(), Unit_Amount_Average]

Wall time: 10.7 s


In [72]:
products.head(5)

,SKU,SKU_Category,Units_Sold,Total_Amount,Unit_Amount_Average
0,00GVC,1TS,2.00,35.36,17.68
1,00OK1,J4R,18.00,22.91,1.27
2,0121I,XG4,31.00,62.74,2.02
3,01IEO,XVK,43.00,268.00,6.23
4,01IQT,Z4O,7.00,9.22,1.32


###  Criando base de dados de clientes

In [ ]:
# 1.1 Para base de clientes, decidi colocar o maximo de informaçoes que podem vir a serem uteis possiveis, como por exemplo
# ... quando foi a primeira e ultima compra do cliente(que aparecem como mesma data caso o cliente tenha comprado apenas 1x),
# ... valores e quantidade de pedidos e produtos comprados
# 1.2 Em um caso similar a base de dados de produtos, decidi agregar colunas de dicionarios de produtos e suas categorias que o
# ... cliente ja comprou assim como as quantidades compradas dos mesmos, que possui menos casos de colunas gigantescas, e
# ... quando se tem esses casos, casos menores

In [76]:
customers = pd.DataFrame(columns=['Customer_ID', 'SKU_Category', 'SKU', 'Average_Transaction_Amount', 'Total_Transactions', 'Total_Products', 'Total_Spent', "First_Order_Date", "Last_Order_Date"])
customers

,Customer_ID,SKU_Category,SKU,Average_Transaction_Amount,Total_Transactions,Total_Products,Total_Spent,First_Order_Date,Last_Order_Date


In [77]:
%%time
for cID, data in df.groupby(['Customer_ID']):
    skus = dict(Counter(data.SKU.to_list()).most_common())
    skucs = dict(Counter(data.SKU_Category.to_list()).most_common())
    ata = data.Sales_Amount.sum() / len(data.Transaction_ID.unique())
    customers.loc[len(customers.index)] = [cID, skucs, skus, ata, len(data.Transaction_ID.unique()), data.Quantity.sum(),
                                           data.Sales_Amount.sum(), data.Date.min(), data.Date.max()]

Wall time: 1min 34s


In [78]:
customers.head(5)

,Customer_ID,SKU_Category,SKU,Average_Transaction_Amount,Total_Transactions,Total_Products,Total_Spent,First_Order_Date,Last_Order_Date
0,1,"{'0H2': 1, 'N8U': 1}","{'6OUVC': 1, 'CEBU8': 1}",16.29,1,2.00,16.29,2016-01-22,2016-01-22
1,2,"{'TVL': 1, 'F9B': 1}","{'2SLS0': 1, 'GZ6VU': 1}",11.38,2,2.00,22.77,2016-03-24,2016-06-19
2,3,"{'TW8': 2, 'LPF': 1}","{'Y1M2E': 1, 'WHV3G': 1, 'WALAE': 1}",10.92,1,4.00,10.92,2016-01-02,2016-01-02
3,4,"{'69B': 1, 'YMJ': 1, '29A': 1, 'N8U': 1, 'JR5'...","{'APEJQ': 1, 'QR49F': 1, 'RBNO8': 1, '89ULV': ...",16.64,2,5.00,33.29,2016-07-11,2016-11-09
4,5,"{'P42': 4, 'LGI': 1}","{'DMJ70': 4, 'EIFZN': 1}",15.76,5,14.00,78.82,2016-02-09,2016-07-05


### Nova base de transaçoes 

In [4]:
df.columns

Index(['Unnamed: 0', 'Date', 'Customer_ID', 'Transaction_ID', 'SKU_Category',
       'SKU', 'Quantity', 'Sales_Amount'],
      dtype='object')

In [5]:
# 1.1 Renomeando coluna de 'index'
# 1.2 Nova base possue tambem Date como atributo pandas que foi tratado antes no codigo
df = df.rename(columns={"Unnamed: 0": 'Transaction_P_ID'})
df.columns

Index(['Transaction_P_ID', 'Date', 'Customer_ID', 'Transaction_ID',
       'SKU_Category', 'SKU', 'Quantity', 'Sales_Amount'],
      dtype='object')

### Salvando novas bases de dados 

In [79]:
customers.to_csv("customers_1.csv", index = False)

In [80]:
# 1.1 Ids de produtos sao strings entao achei pertinente deixar um id numerico
products.to_csv("products_1.csv", index_label = "SKU_N")

In [6]:
df.to_csv("transactions_1.csv", index = False)

## Tentativa base de dados de produtos com coluna de compras conjuntas

In [81]:
# 1.1 O feito abaixo é a verificaçao da viabilidade de sentido de na base criada anteriormente de produtos ter uma coluna com um
# ... dicionario com os produtos que mais sao vendidos juntos com ele e a quantidade de vezes do acontecimento
# 1.2 Para isso foi pego o que *deve* ser o pior cenario possivel que é o produto mais vendido e feita uma visualizaçao de como
# ... ficaria na pratica se a coluna fosse implementada
# 1.3 Pior cenario -> UNJKW(SERA CHAMADO DE PMV - PRODUTO MAIS VENDIDO), que foi vendido mais de 2000 vezes,
# ... com larga vantagem em relaçao ao proximo produto
df.SKU.value_counts()[:5]

UNJKW    2007
COWU2     791
OV1P9     737
M6J9W     698
C6TXL     689
Name: SKU, dtype: int64

In [84]:
# 1.1 Agrupando a base de dados com apenas os dados que tiveram o PMV
MOST_SKU_APRIORI = df[df['SKU'] == 'UNJKW']
MOST_SKU_APRIORI.sample(3)

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
56841,56842,2016-06-07,15256,28284,N8U,UNJKW,1.00,2.11
63970,63971,2016-06-27,17698,31730,N8U,UNJKW,1.00,2.11
75389,75390,2016-08-31,13209,41571,N8U,UNJKW,2.00,4.23


In [87]:
# 1.1 Agrupamento por transaçao, com objetivo de verificar em quais transaçoes o PMV aparece e contar suas compras casadas
transactions_frames = df.groupby(['Transaction_ID'])
len(transactions_frames)

64682

In [88]:
# 1.1 Colocando os ids de transaçoes que o PMV aparece em uma lista para ser iterado
MOST_SKU_APRIORI_TRANSACTIONS = MOST_SKU_APRIORI.Transaction_ID.to_list()

In [89]:
%%time
# 1.1 Adiciono a uma lista dos produtos que foram comprados juntos com ele
dic_test = []
for i in MOST_SKU_APRIORI_TRANSACTIONS:
    dic_test = dic_test + transactions_frames.get_group(i).SKU.to_list()

Wall time: 545 ms


In [90]:
# 1.1 Uso counter para agrupar e contar os produtos
dic_test_counted = Counter(dic_test)

In [46]:
# 1.1 Foram 1143 produtos comprados juntos com o PMV, queria seria completamente sem cabimento colocar em uma celular de uma
# ... base de dados um dicionario de 1143 itens de tamanho
len(dic_test_counted)

1143

In [92]:
# 1.1 Para descartar completamente essa possibilidade pensei em agrupar o dicionario por vezes que tal produto aparece com ele
# ... ou seja, o que antes seria um dicionario de (produto, quantidade) passaria para (quantidade, produto(s)), assim
# ... diminuindo um pouco o tamanho pela repetiçao de apariçao
# 1.2 Ordenando e visualisando os mais comuns (claro que ele proprio seria apagado se fosse utilizado na pratica)
dic_test_counted_ordered = dic_test_counted.most_common()
print(dic_test_counted_ordered[:5])
print(dic_test_counted_ordered[-5:]) # nova agrupaçao diminuiria a quantidade de informaçao desses exemplos principalmente

[('UNJKW', 2007), ('COWU2', 71), ('OV1P9', 57), ('M6J9W', 47), ('UQC6F', 40)]
[('NZR88', 1), ('I1MDT', 1), ('RBNO8', 1), ('MZVSL', 1), ('E6O8G', 1)]


In [93]:
# 1.1 Feita nova agrupaçao e sem surpresas, fica menor mas sem sentido para o problema mesmo assim
# 1.2 Print foi comentada por ser muit longa, mas sinta-se livre para baixar o codigo e ver por si proprio
dic_test_counted_ordered_copy = dic_test_counted_ordered
d = {}
for key, value in dic_test_counted_ordered_copy:
    if value not in d:
        d[value] = [key]
    else:
        d[value].append(key)
#print(d)